In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Marketing Analytics Jumpstart - Recommended Events Analysis

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/marketing-analytics-jumpstart/blob/main/notebooks/events_analysis.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fmarketing-analytics-jumpstart%2Fmain%2Fnotebooks%2events_analysis.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/marketing-analytics-jumpstart/blob/main/notebooks/events_analysis.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/marketing-analytics-jumpstart/main/notebooks/events_analysis.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

* Author: Federico Patota - fpatota@google.com

* Latest revision: January 24, 2025


### Description
This notebook is used to perform an exploratory data analysis (EDA) and generate a data profiling report for Google Analytics 4 (GA4) data stored in BigQuery.

The goal is to support the marketing team in selecting a short list of events to use for building a lead score propensity model.

This notebook also leverages Vertex AI and Gemini to generate an initial recommendation based on the output of the Events Analysis.

### Requirements
- You have a Google Cloud project with BigQuery and Vertex AI APIs enabled.
- You have exported GA4 data to BigQuery using the [integrated connector](https://support.google.com/analytics/answer/9358801?hl=en)
- You have the necessary permissions to access the data and run queries in BigQuery.
- You have already identified a *target event* that you want to use in the propensity model.
- You have good knowledge of data analysis concepts.

Estimated Analysis time is around **30-40 minutes**.

In [ ]:
# @title Step 1 – Installing the required Python packages
# @markdown Click the ( ▶ ) button to ensure that the required packages are installed:
# @markdown * [google-cloud-bigquery](https://pypi.org/project/google-cloud-bigquery/)
# @markdown * [vertexai](https://pypi.org/project/vertexai/)
# @markdown * [pandas](https://pypi.org/project/pandas/)
# @markdown * [numpy](https://pypi.org/project/numpy/)
# @markdown * [scipy](https://pypi.org/project/scipy/)
# @markdown * [ydata-profiling](https://pypi.org/project/ydata-profiling/)
# @markdown * [ipywidgets](https://pypi.org/project/ipywidgets/)

# The packages come pre-installed on Colab and Jupyther Notebooks
import sys, importlib, time
from IPython.display import display, Markdown, clear_output
from IPython import Application

using_google_colab = "google.colab" in sys.modules

def install_packages(package_name_list):
    """
    Installs the specified packages.

    Args:
        package_name_list: A list of package names to be installed.

    Returns:
        True if the package was installed, False otherwise.
    """
    added_packages = False
    already_installed_list = !pip list
    already_installed = " ".join(already_installed_list)
    for package_name in package_name_list:
      if package_name in already_installed:
        print(f"Package {package_name} already present")
      else:
        print(f"Installing {package_name}")
        !pip install {package_name} --quiet
        print("Package installed")
        added_packages = True
      time.sleep(1)
      clear_output(wait=True)
    return added_packages

packages = ["google-cloud-bigquery",
            "vertexai",
            "pandas",
            "numpy",
            "scipy",
            "ydata-profiling",
            "ipywidgets"]

added_packages = install_packages(packages)

# Clearing the display after the install
clear_output(wait=True)

if added_packages:
  output = """### Packages have been installed. Restarting runtime.
  ### Please wait a few seconds and then proceed to the next step."""
  display(Markdown(output))
  Application.instance().kernel.do_shutdown(True)
else:
  output = "### All packages are ready for use. You can proceed to the next step."
  display(Markdown(output))

### All packages are ready for use. You can proceed to the next step.

In [ ]:
# @title Step 2 - Settings and Authentication

# @markdown To run the notebook, the following APIs will be enabled for the
# @markdown  specified Google Cloud project:
# @markdown  * [Vertex AI API](https://cloud.google.com/vertex-ai/docs/reference/rest)
# @markdown  * [BigQuery API](https://cloud.google.com/bigquery/docs/reference/rest)

# @markdown Fill the information below and then click the ( ▶ ) button to update
# @markdown the settings and authenticate you to Google Cloud.

# @markdown ---
# @markdown #### Google Cloud Platform (GCP)
# @markdown Copy the project ID from the "Project Info" card in the console [Dashboard](https://console.cloud.google.com/home/dashboard).
project_id = "your-project-id" #@param {type:"string"}
# @markdown Insert the location to use for Vertex AI. You can find
# @markdown [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations#genai-locations)
# @markdown a list of all possible locations. If a valid location
# @markdown is not specified, `us-central1` will be used.
vertex_ai_location = "us-central1" #@param {type:"string"}
# @markdown ---
# @markdown #### Google Analytics 4 (GA4)
# @markdown For a quick installation, copy the GA4 property ID.
# @markdown You will find it in your GA4 console, under Admin settings.
ga4_property_id = "123456789" #@param {type:"string"}
# @markdown ---

import sys
import time
import vertexai
from google.cloud import bigquery
from IPython.display import clear_output

def authenticate_notebook():
  """
  Authenticates the Google Colab environment.
  """
  if "google.colab" in sys.modules:
    from google.colab import auth, data_table
    print("Authenticating the Colab Notebook environment.")
    auth.authenticate_user()
    print("Enabling the Colab dataframe formatter.")
    data_table.enable_dataframe_formatter()
  else:
    # If not in a Colab, authenticating using gcloud cli
    !gcloud auth login

def set_project_id(project_id):
  """
  Sets the Google Cloud project ID.
  """
  !gcloud config set project {project_id}

def enable_api(api_name):
  """
  Enables a Google Cloud API.
  """
  api_enabled = !gcloud services list | grep {api_name}
  if not api_enabled:
    print(f"Enabling {api_name} API", end="")
    !gcloud services enable {api_name}
    print(" - Done.")
  else:
    print(f"{api_name} API is already enabled.")

def validate_gcp_region(location):
  """
  Checks if the input string is a valid GCP region.
  """
  location_check = !gcloud compute regions list | grep {location}
  if location_check:
    return True
  else:
    return False

def initialize_vertex_ai(project_id, gcp_location):
  """
  Initiates the Vertex AI SDK.
  """
  print(f"Initiating the Vertex AI SDK.", end="")
  vertexai.init(project=project_id, location=vertex_ai_location)
  print(" - Done.")

def initialize_bigquery(project_id):
  """
  Initializes the BigQuery client.
  """
  print("Initiating the BigQuery Client", end="")
  bq_client = bigquery.Client(project=project_id)
  print(" - Done.")
  return bq_client

# --- Main execution block ---
authenticate_notebook()
set_project_id(project_id)
enable_api("bigquery.googleapis.com") # BigQuery API
enable_api("aiplatform.googleapis.com") # VertexAI API

# Validating the provided VertexAI location
if not validate_gcp_region(vertex_ai_location):
  print(f"Invalid VertexAI location. Using default value.")
  vertex_ai_location = "us-central1"

# Initializing VertexAI SDK
initialize_vertex_ai(project_id, vertex_ai_location)
# Initializing the BQ Client
bq_client = initialize_bigquery(project_id)

# When importing data from GA4 to BigQuery the default connector uses
# a specific naming convention for datasets and tables. Specifically
# the id of the dataset uses the GA4 property ID as a suffix and the
# table containing the GA4 events is named using the pattern
# 'events_YYYYMMDD'. Using 'events_*' as table ID allow us to query
# the right table regardless of the import date.

dataset_id_suffix = ga4_property_id
location = ""
dataset_id = ""
table_id = "events_*"

# Iterate through datasets and find the one with the matching suffix
for dataset in bq_client.list_datasets():
    dataset_id = dataset.dataset_id
    if dataset_id.endswith(dataset_id_suffix):
        dataset_ref = bq_client.get_dataset(dataset.reference)
        location = dataset_ref.location
        print(f"GA4 Dataset ID: {dataset_id}, Location: {location}")
        time.sleep(1)
        clear_output(wait=True)
        display(Markdown("### Done. You can proceed to the next step."))
        break
else:
    time.sleep(1)
    clear_output(wait=True)
    print(f"No dataset found with ID suffix: {dataset_id_suffix}")

### Done. You can proceed to the next step.

In [ ]:
# @title Step 3 - Importing GA4 data from BigQuery

import markdown
from google.cloud import bigquery
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import datetime
import time
import math

def run_bq_job_with_progress(bq_client: bigquery.Client,
                             query: str) -> bigquery.job.query.QueryJob:
  """
  Runs a BigQuery job while displaying elapsed time.
  """
  job = bq_client.query(query)
  start = time.time()
  print("|", end="")
  count = 0
  while job.running():
    count += 1
    # Printing the elapsed time approximately every [40 x 0.5] = 20 seconds
    if count % 40 == 0:
      elapsed_time = time.time() - start
      minutes = str(math.floor(elapsed_time/60)).zfill(2)
      seconds = str(round(elapsed_time%60)).zfill(2)
      print(f"-| {minutes}m:{seconds}s elapsed.")
      print("|", end="")
    else:
      print("-",end="")
    # Iterating every 0.5 seconds
    time.sleep(0.5)
  print("-> Done!")
  job_duration = time.time() - start
  minutes = str(math.floor(job_duration/60)).zfill(2)
  seconds = str(round(job_duration%60)).zfill(2)
  print(f"The job duration was {minutes}m:{seconds}s.")
  return job

def get_event_names() -> list:
  """
  Retrieves a list of event names from a BigQuery table.
  """
  global table_ref
  # SQL query to fetch event names
  event_list_query = f"""
    SELECT DISTINCT event_name
    FROM `{table_ref}`
    ORDER BY event_name DESC
  """

  # Execute the query
  print(f"Retrieving the list of events from {table_ref}.")
  print("This takes on average less than 1 minute.")

  job = run_bq_job_with_progress(bq_client, event_list_query)

  event_names = [row.event_name for row in job.result()]
  return event_names

def generate_events_checkboxes(confirm_target_button):
  """
  Generates a list of checkboxes for the available events.
  """
  global target_dropdown_widget, events_checkboxes, target
  global confirm_features_button
  target = target_dropdown_widget.value
  if not target:
    print("Please select a target event.")
    return

  # Clearing the output in case the confirm target button gets clicked again
  clear_output(wait=True)

  output = """
  ## Events Selection

  ### Select a target event and confirm your selection.
  """

  hbox = widgets.HBox([target_dropdown_widget, confirm_target_button])
  display(Markdown(output), hbox)

  events_checkboxes = []

  # Create checkboxes for event selection
  for event_name in event_names:
    if event_name != target:
      events_checkboxes.append(
          widgets.Checkbox(value=False, description=event_name, indent=False))

  confirm_features_button.on_click(process_events_data)

  grid = widgets.GridBox(events_checkboxes,
                         layout=widgets.Layout(
                             grid_template_columns=f"repeat(3, 1fr)"))

  output = """
  ### Select 1 or more feature events and confirm your selection.
  """

  display(Markdown(output), confirm_features_button, grid)

def build_etl_query() -> str:
  """
  Builds the ETL query to be executed in BigQuery.
  """
  global table_ref, start_date, end_date, rolling_window, sampling_percent
  global random_sampling, limit_rows, target, selected_events

  start_date_sql = start_date.strftime("%Y%m%d")
  end_date_sql = end_date.strftime("%Y%m%d")

  features_with_target = [target] + selected_events

  etl_query = f"""
  CREATE OR REPLACE TEMP TABLE dates_interval as (
    SELECT DISTINCT
    -- Select each distinct event_date as 'input_date',
    -- representing the current date in the analysis
    PARSE_DATE('%Y%m%d',event_date) as input_date,
    -- Calculate the 'end_date' by subtracting a specified
    -- interval from the 'input_date'
    DATE_SUB(PARSE_DATE('%Y%m%d',event_date),
     INTERVAL {rolling_window} DAY) as end_date
    FROM `{table_ref}`
    -- The time interval of {date_range_length} days
    WHERE event_date BETWEEN '{start_date_sql}' AND '{end_date_sql}'
    ORDER BY input_date DESC
  );

  CREATE OR REPLACE TEMP TABLE events_users as (
    SELECT DISTINCT
      -- User identifier
      Users.user_pseudo_id,
      -- Date for which the feature is being calculated
      DI.input_date as feature_date
    FROM `{table_ref}` Users
    CROSS JOIN dates_interval as DI
    WHERE PARSE_DATE('%Y%m%d',Users.event_date)
      BETWEEN DI.end_date AND DI.input_date
  );
  """

  outer_join_string = ""
  coalesce_string = ""

  for count, event in enumerate(features_with_target):
    etl_query += f"""
    CREATE OR REPLACE TEMP TABLE rolling_{event}_past_days AS (
      SELECT user_pseudo_id, input_date as feature_date,
        -- Number of distinct {event} events the user generated
        -- in the past {rolling_window} days.
        MAX(COUNT(DISTINCT CASE DATE_DIFF(input_date,
          PARSE_DATE('%Y%m%d',event_date), DAY) BETWEEN 1 AND {rolling_window}
           WHEN TRUE THEN event_timestamp END))
            OVER(PARTITION BY user_pseudo_id, input_date) AS {event}_past_{rolling_window}_days
      FROM `{table_ref}` as E
      CROSS JOIN dates_interval as DI
      -- Filter events in the specified date range
      WHERE PARSE_DATE('%Y%m%d',E.event_date)
          BETWEEN DI.end_date AND DI.input_date
        -- Consider only events of the specific type
        AND event_name='{event}'
        -- Grouping by user_pseudo_id and feature_date
      GROUP BY user_pseudo_id, feature_date
    );
    """

    coalesce_string += f"COALESCE(T{count}.{event}_past_{rolling_window}_days,\
     0) AS {event}_past_{rolling_window}_days,"

    outer_join_string += f"""
      FULL OUTER JOIN rolling_{event}_past_days AS T{count}
      ON EUD.user_pseudo_id = T{count}.user_pseudo_id
        AND EUD.feature_date = T{count}.feature_date"""

  etl_query += f"""
  CREATE OR REPLACE TEMP TABLE etl_result AS (
  SELECT DISTINCT
    -- Date for which the features are calculated
    EUD.feature_date,
    -- User identifier
    EUD.user_pseudo_id,
    {coalesce_string}
  FROM events_users AS EUD
  -- This performs a full outer join, which combines all rows from both tables, including those that don't have matching values.
  {outer_join_string}
  -- This filters the results to include only rows where the user_pseudo_id is not null
  WHERE EUD.user_pseudo_id IS NOT NULL
  ); -- Ordering by the target feature

  SELECT * EXCEPT (feature_date, user_pseudo_id) FROM etl_result
  TABLESAMPLE SYSTEM ({sampling_percent} PERCENT)
  WHERE rand() < {random_sampling}
  LIMIT {limit_rows};
  """

  return etl_query

# Function to handle button click and build follow-up query
def process_events_data(confirm_features_button):
  global table_ref, start_date, end_date, rolling_window, target
  global selected_events, events_checkboxes, bqdf

  # Adding the selected features in a list
  selected_events= [
      checkbox.description for checkbox in events_checkboxes if checkbox.value]

  # At least an event should be selected
  if not selected_events:
    print("Please select at least one event to be used for features.")
    return

  time.sleep(1)
  clear_output(wait=True)

  print(f"Selected target event: {target}")
  print(f"Selected events to be used for features: {selected_events}")

  features_count = len(selected_events)

  # Estimating the query execution time
  best_mins = round((1+features_count)*0.5) # Best case
  worst_mins = 1+features_count # Worst case

  print(f"The processing should take between {best_mins} and {worst_mins} minutes.")

  etl_query = build_etl_query()

  # Execute the query
  job = run_bq_job_with_progress(bq_client, etl_query)

  time.sleep(1)
  clear_output(wait=True)

  print("Saving the results in a dataframe.")
  bqdf = job.to_dataframe(progress_bar_type='tqdm_notebook')

  time.sleep(1)
  clear_output(wait=True)

  output_prefix = f"""
  ### Data extraction and transformation completed
  * Selected target event: *{target}*
  * Selected {len(selected_events)} feature events that will be used for analysis:
   ***{'***, ***'.join(selected_events)}***
  """
  output_suffix = """
  ### You can proceed to the next step.
  """

  display(Markdown(output_prefix), Markdown(output_suffix))

# --- Main execution block --- #

end_date = datetime.datetime.today()
date_range_length = 30
start_date = end_date - datetime.timedelta(days=date_range_length)
rolling_window = 7

# @markdown Because the amount of data generated by GA4 is very large
# @markdown and not ready for exploration, some data selection and aggregation
# @markdown are needed.

# @markdown ---
# @markdown #### Filtering and Aggregating GA4 Data
# @markdown By default, only the data from the last 30 days will be used,
# @markdown aggregated using 7-days rolling windows.
# @markdown To change this setting, tick the checkbox below and select a
# @markdown different date interval and rolling window length.

use_custom_range = False # @param {"type":"boolean"}
custom_start_date = "2024-01-24" # @param {"type":"date"}
custom_end_date = "2025-01-24" # @param {"type":"date"}
custom_rolling_window = 30 # @param {"type":"integer"}

# @markdown The default settings will be applied regardless of the checkbox if:
# @markdown * The specified date range is smaller than the rolling window.
# @markdown * The difference between end date and start date is 1 day or less.
# @markdown * The specified rolling window value is less than 1.
# @markdown * The specified end date is in the future.

# Converting the dates to datetime objects
custom_start_date = datetime.datetime.strptime(custom_start_date, "%Y-%m-%d")
custom_end_date = datetime.datetime.strptime(custom_end_date, "%Y-%m-%d")
custom_date_range_length = (custom_end_date - custom_start_date).days

if use_custom_range:
  try:
    if custom_end_date > datetime.datetime.today():
      print("The start date is in the future.")
      # End date in the future
      raise ValueError
    if custom_date_range_length <= 1:
      # Range shorter than 1 day
      print("The specified date range is less than 1 day.")
      raise ValueError
    if custom_date_range_length < custom_rolling_window:
      # Range shorter than the rolling window
      print("The specified date range is shorter than the rolling window.")
      raise ValueError
    if custom_rolling_window < 1:
      # Rolling window too small
      print("The specified rolling window is less than 1 day.")
      raise ValueError
    start_date = custom_start_date
    end_date = custom_end_date
    date_range_length = custom_date_range_length
    rolling_window = custom_rolling_window
  except ValueError:
    print("The specified custom date range is invalid. Using default values.")

print(f"""
Date range: {start_date.strftime("%Y-%m-%d")} - {end_date.strftime("%Y-%m-%d")}.
Date range length: {date_range_length} day{'s' if rolling_window > 1 else ''}.
Rolling window length: {rolling_window} day{'s' if rolling_window > 1 else ''}.
""")

# @markdown ---

# @markdown #### Event Selection
# @markdown 1. Click the ( ▶ ) button to import the list of available events
# @markdown from BigQuery. This should take approximately 30 seconds.
# @markdown 2. When the import is complete, a drop-down menu and a
# @markdown "Confirm Target" button will appear. Select the target event from
# @markdown the drop-down menu and click "Confirm Target".
# @markdown 3. A list of checkboxes for all the remaining events will appear.
# @markdown Tick the checkboxes of the events that you want to include in the
# @markdown exploratory data analysis and click the "Confirm Features" button.
# @markdown 4. Wait for the execution to complete. This step should take between
# @markdown 30 and 60 seconds for each selected event (including the target).

# Construct the full table ID
table_ref = f"{project_id}.{dataset_id}.{table_id}"
# The variable that will contain the target event
target = None
# Button for confirming dropdown target feature
confirm_target_button = widgets.Button(description="Confirm Target")
# Button for confirming the selected features
confirm_features_button = widgets.Button(description="Confirm Features")
# List that will contain the checkboxes to select the events used for features
events_checkboxes = []
# Lists containing only the selected events that will be processed as features
selected_events= []
# Sampling percent (via tablesample method)
sampling_percent = 10
# Random subsampling
random_sampling = 0.1
# Limit of rows to load in the dataframe
limit_rows = 50000
# The dataframe containing the results
bqdf = None

event_names = get_event_names()

time.sleep(1)
clear_output(wait=True)

output = """
### Events Selection

Select a target event and confirm your selection
"""

# Dropdown widget for target event selection
target_dropdown_widget = widgets.Dropdown(
    options=event_names,
    description='Target:',
    disabled=False,
)

confirm_target_button.on_click(generate_events_checkboxes)

hbox = widgets.HBox([target_dropdown_widget, confirm_target_button])
display(Markdown(output), hbox)


  ### Data extraction and transformation completed
  * Selected target event: *purchase*
  * Selected 7 feature events that will be used for analysis:
   ***view_promotion***, ***view_cart***, ***user_engagement***, ***select_promotion***, ***add_to_wishlist***, ***add_to_cart***, ***add_shipping_info***  
  


  ### You can proceed to the next step.
  

In [ ]:
# @title Step 4 - Generating a data profiling report with ydata_profiling

# @markdown ##### 1. **Data Sampling**
# @markdown Use the slider to select the maximum number of
# @markdown rows to include in the report (max 50,000).  A larger sample size
# @markdown  may increase processing time. Suggested value: `10000`.
max_rows = 10000 # @param {"type":"slider","min":100,"max":50000,"step":100}

# @markdown ##### 2. **Report Filename**
# @markdown You can edit this if you want a different
# @markdown filename for your data profiling report.
# @markdown Suggested value `profiling_report`.
filename = "profiling_report" #@param {type:"string"}

# @markdown ##### 3. **Report Generation**
# @markdown Click the ( ▶ ) button to create the report.

# @markdown ##### 4. **Download**
# @markdown Once the report is ready, click the
# @markdown "Download Report" button that will appear to save it locally.

from ydata_profiling import ProfileReport
import ipywidgets as widgets
from IPython.display import display
import sys, base64


# Renaming the first column to highlight the fact that
# it's the target

target_column = f"Target: {bqdf.columns[0]}"

# Subsampling (if needed)
sample = bqdf.rename(columns={bqdf.columns[0]: target_column})
if len(bqdf) > max_rows:
  print(f"""As the dataframe has {len(bqdf)} rows, a random
   sub-sampling of {max_rows} will be performed.""")
  sample = sample.sample(max_rows, random_state=42)

# Generating the profiling report for the data
profile = ProfileReport(sample, title="Data Profiling Report", minimal=False)
profile.to_file(f"{filename}.html")

# Creating a button widget
download_button = widgets.Button(description="Download Report")

# Function to handle the download when the button is clicked
def download_report(button):
  if "google.colab" not in sys.modules:
    from google.colab import files
    files.download(f"{filename}.html")
  else:
    html_output = widgets.HTML(markdown.markdown(output))
    from IPython.display import FileLink
    file_link = FileLink(f"{filename}.html")
    display(html_output, file_link)

time.sleep(1)
clear_output(wait=True)

output_prefix = """
### Data Profiling Report

To download the data profiling report, click on the button below:
"""
output_suffix = """### Report downloaded. You can proceed to the next step."""

file_path = f"{filename}.html"
payload = base64.b64encode(open(file_path, 'rb').read()).decode('utf-8')

html_button = f"""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script> </head>
<body>
<a download="{filename}.html" href="data:text/html;base64,{payload}" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button">Download Report</button>
</a>
<div id="message"></div>

<script>
  const button = document.querySelector('button');
  const messageDiv = document.getElementById('message');

  button.addEventListener('click', () => {{
    const markdownText = `{output_suffix}`;
    messageDiv.innerHTML = marked.parse(markdownText);
  }});
</script>

</body>
</html>
"""

# Creating the download button (via HTML)
download_button = widgets.HTML(html_button)

display(Markdown(output_prefix), download_button)


### Data Profiling Report

To download the data profiling report, click on the button below:


HTML(value='<html>\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<script src=…

In [ ]:
# @title Step 5 - Performing an exploratory data analysis (EDA)
# @markdown Click the ( ▶ ) button to perform an exploratory data analysis and
# @markdown display the results.

# @markdown We are using Pandas, Numpy, and Scipy to perform an exploratory data
# @markdown analysis.

# @markdown The output of this step will be provided as context to an LLM to
# @markdown to be able to generate tailored recommendations.

# @markdown For each column are computed:
# @markdown * Descriptive statistics (max, min, top, count, quartiles, etc.)
# @markdown * Statistical relationship with the target variable (Cramer's V,
# @markdown ANOVA, Kruskal-Wallis, correlation)

# @markdown The statistical methods and functions change depending on the type
# @markdown of each feature column and the target column, whether they are
# @markdown categorical or numerical.

# @markdown Correlation and covariance matrices are computed for
# @markdown all the numerical columns.


import pandas as pd
from scipy.stats import chi2_contingency, f_oneway, kruskal
import numpy as np
from IPython.display import display, Markdown


def cramers_v(x, y):
    """
    Calculates Cramer's V for two categorical variables.
    This is a measure of association between two categorical variables,
    ranging from 0 (no association) to 1 (perfect association).
    """
    contingency_table = pd.crosstab(x, y)
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    phi2 = chi2 / n
    r, k = contingency_table.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

def analyze_relationship(df, target_variable, col):
    """
    Analyzes the relationship between a column and the target variable
    based on their data types.

    For categorical columns, it provides:
    - Count
    - Number of unique values
    - Most frequent value and its frequency
    - Cramer's V with the target variable (if the target is also categorical)
    - ANOVA F-statistic and p-value (if the target is numerical)

    For numerical columns, it provides:
    - Count
    - Mean
    - Standard deviation
    - Minimum and maximum values
    - Quartiles
    - Kruskal-Wallis H-statistic and p-value (if the target is categorical)
    - Correlation with the target variable (if the target is also numerical)
    """
    summary = df.describe(include="all")

    summary_markdown_str = ""

    col_type = summary.dtypes[col]

    if col_type == 'object':
      # Categorical column
      summary_markdown_str += f"\n### {col} (Categorical)"  # Markdown heading
      if col == target_variable:
        summary_markdown_str += " - This is the target variable"
      summary_markdown_str += f"\n* Type: {col_type}\n"  # Markdown bullet
      summary_markdown_str += f"* Count: {summary.loc[:, col]['count']}\n"
      summary_markdown_str += f"* Unique Values: {summary.loc[:, col]['unique']}\n"
      summary_markdown_str += f"* Most Frequent Value: {summary.loc[:, col]['top']}\n"
      summary_markdown_str += f"* Frequency of Most Frequent Value: {summary.loc[:, col]['freq']}\n"
      if df[target_variable].dtype == 'object':
        # Target is also categorical: Use Cramer's V
        cramers_v_value = cramers_v(df[col], df[target_variable])
        summary_markdown_str += f"* Cramer's V with {target_variable}: {cramers_v_value}\n"
      else:
        # Target is numerical: Use ANOVA
        groups = [df[target_variable][df[col] == cat] for cat in df[col].unique()]
        f_statistic, p_value = f_oneway(*groups)
        summary_markdown_str += f"* ANOVA F-statistic with {target_variable}: {f_statistic}\n"
        summary_markdown_str += f"* P-value: {p_value}\n"
    elif col_type in ['Int64', 'Float64']:
      # Numerical column
      summary_markdown_str += f"\n### {col} (Numerical)"  # Markdown heading
      if col == target_variable:
        summary_markdown_str += " - This is the target variable"
      summary_markdown_str += f"\n* Type: {col_type}\n"  # Markdown bullet
      summary_markdown_str += f"* Count: {summary.loc[:, col]['count']}\n"
      summary_markdown_str += f"* Mean: {summary.loc[:, col]['mean']}\n"
      summary_markdown_str += f"* Standard Deviation: {summary.loc[:, col]['std']}\n"
      summary_markdown_str += f"* Minimum: {summary.loc[:, col]['min']}\n"
      summary_markdown_str += f"* 25th Percentile: {summary.loc[:, col]['25%']}\n"
      summary_markdown_str += f"* 50th Percentile (Median): {summary.loc[:, col]['50%']}\n"
      summary_markdown_str += f"* 75th Percentile: {summary.loc[:, col]['75%']}\n"
      summary_markdown_str += f"* Maximum: {summary.loc[:, col]['max']}\n"
      if df[target_variable].dtype == 'object':
        # Target is categorical: Use Kruskal-Wallis or ANOVA if assumptions are met
        groups = [df[col][df[target_variable] == cat] for cat in df[target_variable].unique()]
        if len(groups) > 2: # Kruskal-Wallis for 3 or more groups
          h_statistic, p_value = kruskal(*groups)
          summary_markdown_str += f"* Kruskal-Wallis H-statistic with {target_variable}: {h_statistic}\n"
          summary_markdown_str += f"* P-value: {p_value}\n"
        else: # ANOVA for 2 groups
          f_statistic, p_value = f_oneway(*groups)
          summary_markdown_str += f"* ANOVA F-statistic with {target_variable}: {f_statistic}\n"
          summary_markdown_str += f"* P-value: {p_value}\n"
      else:
        # Target is numerical: Use correlation
        correlation = df[col].corr(df[target_variable])
        summary_markdown_str += f"* Correlation with {target_variable}: {correlation}\n"

    summary_markdown_str += "\n"  # Add an extra newline for better readability
    return summary_markdown_str

# --- Main execution block ---

# Initialize the output string with a heading
analysis_output = f"## Exploratory Data Analysis – {target_column}"


no_type_error = True

# Loop through each column in the DataFrame
for count, col in enumerate(sample.columns, start=1):

# Analyze the relationship between the current column and the target variable
  try:
    print(f"Analyzing column {count}/{len(sample.columns)} - {col}", end="")
    analysis_output += analyze_relationship(sample, target_column, col)
    print(f" – Done.")
  except TypeError:
    print(f" – Skipped due to TypeError.")
    no_type_error = False

# Add correlation and covariance matrices to the output
analysis_output += "\n ## Correlation Matrix\n\n"
analysis_output += sample.corr(numeric_only=True).to_markdown()
analysis_output += "\n \n ## Covariance Matrix\n\n"
analysis_output += sample.cov(numeric_only=True).to_markdown()

# If there were errors when analyzing, we leave the issues displayed
if no_type_error:
  time.sleep(1)
  clear_output(wait=True)

output = """
### Exploratory data analysis completed. You can proceed to the next step.

If you want to display the results, click on the button below:
"""

display_analysis_button = widgets.Button(description="Display Analysis")

def display_analysis(button):
  time.sleep(1)
  clear_output(wait=True)
  display(Markdown(analysis_output))
  output = """
  ### Analysis displayed. You can proceed to the next step.
  """
  display(Markdown(output))

display_analysis_button.on_click(display_analysis)

display(Markdown(output), display_analysis_button)

## Exploratory Data Analysis – Target: purchase_past_30_days
### Target: purchase_past_30_days (Numerical) - This is the target variable
* Type: Float64
* Count: 10000.0
* Mean: 0.0123
* Standard Deviation: 0.11022669818378508
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 1.0
* Correlation with Target: purchase_past_30_days: 1.0


### view_promotion_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.5889
* Standard Deviation: 0.5585051322935894
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 1.0
* 75th Percentile: 1.0
* Maximum: 2.0
* Correlation with Target: purchase_past_30_days: 0.0204127880773618


### view_cart_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.0656
* Standard Deviation: 0.4160696472889359
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 19.0
* Correlation with Target: purchase_past_30_days: 0.4294426022110363


### user_engagement_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.6819
* Standard Deviation: 1.1522348521527421
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 1.0
* Maximum: 23.0
* Correlation with Target: purchase_past_30_days: 0.29223845130267745


### select_promotion_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.0686
* Standard Deviation: 0.2722158153837034
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 5.0
* Correlation with Target: purchase_past_30_days: 0.02853833605672141


### add_to_wishlist_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.0036
* Standard Deviation: 0.13707267740833395
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 8.0
* Correlation with Target: purchase_past_30_days: 0.030165034480866937


### add_to_cart_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.0966
* Standard Deviation: 0.6288942604278401
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 18.0
* Correlation with Target: purchase_past_30_days: 0.4315407785295083


### add_shipping_info_past_30_days (Numerical)
* Type: Float64
* Count: 10000.0
* Mean: 0.0404
* Standard Deviation: 0.2904759501710301
* Minimum: 0.0
* 25th Percentile: 0.0
* 50th Percentile (Median): 0.0
* 75th Percentile: 0.0
* Maximum: 8.0
* Correlation with Target: purchase_past_30_days: 0.5748267448399595


 ## Correlation Matrix

|                                |   Target: purchase_past_30_days |   view_promotion_past_30_days |   view_cart_past_30_days |   user_engagement_past_30_days |   select_promotion_past_30_days |   add_to_wishlist_past_30_days |   add_to_cart_past_30_days |   add_shipping_info_past_30_days |
|:-------------------------------|--------------------------------:|------------------------------:|-------------------------:|-------------------------------:|--------------------------------:|-------------------------------:|---------------------------:|---------------------------------:|
| Target: purchase_past_30_days  |                       1         |                     0.0204128 |                0.429443  |                      0.292238  |                       0.0285383 |                      0.030165  |                  0.431541  |                        0.574827  |
| view_promotion_past_30_days    |                       0.0204128 |                     1         |                0.0248249 |                      0.0207146 |                       0.24669   |                      0.016721  |                  0.0188273 |                        0.0228611 |
| view_cart_past_30_days         |                       0.429443  |                     0.0248249 |                1         |                      0.327451  |                       0.0962463 |                      0.0151482 |                  0.594192  |                        0.777433  |
| user_engagement_past_30_days   |                       0.292238  |                     0.0207146 |                0.327451  |                      1         |                       0.135262  |                      0.0344796 |                  0.311952  |                        0.319579  |
| select_promotion_past_30_days  |                       0.0285383 |                     0.24669   |                0.0962463 |                      0.135262  |                       1         |                      0.0523468 |                  0.0956505 |                        0.0598066 |
| add_to_wishlist_past_30_days   |                       0.030165  |                     0.016721  |                0.0151482 |                      0.0344796 |                       0.0523468 |                      1         |                  0.0214888 |                        0.0139294 |
| add_to_cart_past_30_days       |                       0.431541  |                     0.0188273 |                0.594192  |                      0.311952  |                       0.0956505 |                      0.0214888 |                  1         |                        0.512959  |
| add_shipping_info_past_30_days |                       0.574827  |                     0.0228611 |                0.777433  |                      0.319579  |                       0.0598066 |                      0.0139294 |                  0.512959  |                        1         |
 
 ## Covariance Matrix

|                                |   Target: purchase_past_30_days |   view_promotion_past_30_days |   view_cart_past_30_days |   user_engagement_past_30_days |   select_promotion_past_30_days |   add_to_wishlist_past_30_days |   add_to_cart_past_30_days |   add_shipping_info_past_30_days |
|:-------------------------------|--------------------------------:|------------------------------:|-------------------------:|-------------------------------:|--------------------------------:|-------------------------------:|---------------------------:|---------------------------------:|
| Target: purchase_past_30_days  |                     0.0121499   |                    0.00125666 |              0.0196951   |                      0.0371163 |                     0.000856306 |                    0.000455766 |                 0.0299148  |                      0.0184049   |
| view_promotion_past_30_days    |                     0.00125666  |                    0.311928   |              0.00576874  |                      0.0133304 |                     0.0375052   |                    0.00128009  |                 0.00661292 |                      0.00370881  |
| view_cart_past_30_days         |                     0.0196951   |                    0.00576874 |              0.173114    |                      0.156983  |                     0.0109009   |                    0.000863926 |                 0.155479   |                      0.0939592   |
| user_engagement_past_30_days   |                     0.0371163   |                    0.0133304  |              0.156983    |                      1.32765   |                     0.0424259   |                    0.0054457   |                 0.226051   |                      0.106962    |
| select_promotion_past_30_days  |                     0.000856306 |                    0.0375052  |              0.0109009   |                      0.0424259 |                     0.0741015   |                    0.00195324  |                 0.0163749  |                      0.00472903  |
| add_to_wishlist_past_30_days   |                     0.000455766 |                    0.00128009 |              0.000863926 |                      0.0054457 |                     0.00195324  |                    0.0187889   |                 0.00185243 |                      0.000554615 |
| add_to_cart_past_30_days       |                     0.0299148   |                    0.00661292 |              0.155479    |                      0.226051  |                     0.0163749   |                    0.00185243  |                 0.395508   |                      0.0937067   |
| add_shipping_info_past_30_days |                     0.0184049   |                    0.00370881 |              0.0939592   |                      0.106962  |                     0.00472903  |                    0.000554615 |                 0.0937067  |                      0.0843763   |


  ### Analysis displayed. You can proceed to the next step.
  

In [ ]:
# @title Step 6 - Getting recommendations from Gemini

# @markdown Click the ( ▶ ) button to generate a feature selection
# @markdown recommendation with Gemini, based on the exploratory data
# @markdown analysis performed at Step 5.

# @markdown Gemini should also generate and highlight a short list of features
# @markdown that you can copy-paste in the `terraform.tfvars` file when
# @markdown installing the Marketing Analytics Jumpstart following the
# @markdown [installation guide](https://github.com/GoogleCloudPlatform/marketing-analytics-jumpstart/blob/main/infrastructure/terraform/README.md#manual-installation-of-terraform-modules).


# @markdown ---
# @markdown #### Gemini Parameters
# @markdown The sliders below allow to customize the parameters used with the
# @markdown LLM model to generate the response:

# @markdown 1. **Temperature** controls the randomness in token
# @markdown selection. A lower temperature is good when you expect a true or
# @markdown correct response. A temperature of 0 means the highest probability
# @markdown token is usually selected.
# @markdown A higher temperature can lead to diverse or unexpected results.
# @markdown Suggested value: `0.9`.
temperature = 0.9 # @param {"type":"slider","min":0,"max":2,"step":0.1}

# @markdown 2. **Top-P** changes how the model selects tokens for output.
# @markdown Tokens are selected from most probable to least until the sum of
# @markdown their probabilities equals the top-p value. For example, if tokens
# @markdown A, B, and C have a probability of .3, .2, and .1 and the top-p
# @markdown  value is .5, then the model will select either A or B as the next
# @markdown token (using temperature). For the least variable results, set top-P
# @markdown to 0.
# @markdown Suggested value: `0.95`.
top_p = 0.95 # @param {"type":"slider","min":0,"max":1,"step":0.01}

# @markdown 3. **Top-K** specifies the number of candidate tokens when the model
# @markdown is selecting an output token. Use a lower value for less random
# @markdown responses and a higher value for more random responses. Suggested
# @markdown value: `1`.
top_k = 1 # @param {"type":"slider","min":1,"max":40,"step":1}

# @markdown 4. **Max Output Tokens** determines the maximum amount of text output
# @markdown from one prompt. A token is approximately four characters.
# @markdown Suggested value: `8192`.
max_output_tokens = 8192 # @param {"type":"slider","min":1,"max":8192,"step":1}

# @markdown ---

import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part

from IPython.display import display, Markdown

# Initiating Vertex AI SDK
vertexai.init(project=project_id, location=vertex_ai_location)

# Prepare the prompt for Gemini
prompt = f"""
You are an expert data scientist and AI assistant helping to build a propensity model.

The data comes from an export of GA4 data to BigQuery and we want to select the best events to use to create features to be used in the propensity model.

Here is the output of the feature analysis. We calculated the correlation coefficient between each feature and the target {target}:

{analysis_output}

The features and target were engineered by counting the number of times each event occurred within a {rolling_window}-day rolling window and over a date range of {date_range_length} days. Both these parameters can be adjusted and the exploratory data analysis can be performed again.
If you thing the data we have in the output of the feature analysis is not sufficient to make a recommendation, suggest to re-run the analysis proposing new values for number of days for the rolling windows and the date range.

The event names can be deduced from the feature by removing the '_past_{rolling_window}_days' suffix.

Based on this analysis, which events would you suggest to select for the propensity model?

Start your answer with a title and divide it in sections using markdown.

Provide your suggestions in a clear and concise format, explaining your reasoning. You can organize the selected events in a way that you think is most helpful. For example, you might group them by event type or by importance.

List the suggested events to be used for features and the event to be used as target in the following format:

* Target Event: `target_event`
* Analyzed Events: `event_1`, `event_2`, ..., `event_n`
* Suggested Events: `event_3`, `event_7`, `event_4`, `event_2`

At the end, explicitly provide the following message to help the user copy-paste the suggested events including the list of suggested events:


### Copy the list below and paste into the `terraform.tfvars` file when installing the Marketing Analytics Jumpstart:

```
["event_3", "event_7", "event_4" , "event_2"]
```
"""

model=GenerativeModel("gemini-1.5-pro")
generation_config=GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    max_output_tokens=max_output_tokens
)

response = model.generate_content(prompt, generation_config=generation_config)
display(Markdown(response.text))

## Propensity Model Feature Selection: GA4 Events

This analysis aims to identify the most relevant GA4 events for a propensity model predicting **purchase** events. The current features are based on a 30-day rolling window over a 366-day period.

### Understanding the Analysis

* **Correlation Coefficient:**  Measures the strength and direction of the linear relationship between each event feature and the target variable (`purchase_past_30_days`). Higher absolute values indicate stronger relationships.
* **Covariance Matrix:**  Provides similar information to correlation but on the original scale of the data.  

### Suggested Events

Based on the provided correlation analysis, here are the suggested events for the propensity model, ranked by their correlation with purchase events:

* **Target Event:** `purchase`
* **Analyzed Events:** `view_promotion`, `view_cart`, `user_engagement`, `select_promotion`, `add_to_wishlist`, `add_to_cart`, `add_shipping_info`
* **Suggested Events:** `add_shipping_info`, `view_cart`, `add_to_cart`, `user_engagement`

**Reasoning:**

1. **`add_shipping_info`:** Exhibits the strongest positive correlation (0.575) with purchases. This strongly suggests that users who provide shipping information are much more likely to complete a purchase.

2. **`view_cart`:**  Shows a substantial positive correlation (0.429) with purchases. Viewing the cart indicates a clear intent to purchase. 

3. **`add_to_cart`:** Also displays a strong positive correlation (0.432) with purchases.  Adding items to the cart is a strong indicator of purchase intent.

4. **`user_engagement`:** While the correlation is moderate (0.292), it's worth including as it provides a broader picture of user activity. Higher engagement might generally lead to higher conversion rates.

**Important Considerations:**

* **Causation vs. Correlation:** While these events are strongly correlated with purchases, correlation does not equal causation. Further analysis and domain expertise are needed to understand the underlying reasons behind these relationships. 
* **Feature Engineering:** Experiment with different aggregations (e.g., total count, frequency, recency) of these events over various time windows (e.g., 7-day, 14-day) to potentially improve the model's predictive power.

### Copy the list below and paste into the `terraform.tfvars` file when installing the Marketing Analytics Jumpstart:

```
["add_shipping_info", "view_cart", "add_to_cart", "user_engagement"]
``` 
